<a id="content"><h3>Content</h3></a>

- [Dependencies, paths and datasets upload](#upload)
- [Raw dataset consolidation](#raw):
    - [MGI-MP associations consolidation from HMD and PhenoGeno](#mouse)
    - [Human phenotypes dataset consolidation](#human)
    - [Human and mouse datasets combining](#humandmouse)
- [Up-leveling HPO and MPO terms using HPO-MPO system level term mapping by MGI](#uplevel)
- [Filtering the dataset to monosemantic](#mono)
- [Low-level MPO and HPO terms filtering](#lowfilter)

<a id="upload"><h4>Dependencies, paths and datasets upload</h4></a>
[Back to content](#content)

In [1]:
#!/usr/bin/env python3

from collections import defaultdict
from functools import reduce
from operator import or_
import itertools
import os

import pandas as pd

from low2system import replace_terms_by_level
from ontobio.ontol_factory import OntologyFactory

In [2]:
# set paths to databases and ontologies

HMD = "../databases/HMD.rpt"
PHENO_GENO = "../databases/PhenoGeno.rpt"
HPO = "../databases/HPO.txt"
HP2MP = "../databases/HP2MP.tsv"

PATH_HPO = os.path.abspath("../ontologies/hpo.json")
PATH_MPO = os.path.abspath("../ontologies/mpo.json")

<a id="raw"><h4>Raw dataset consolidation</h4></a>

Consolidating raw database with low-level HPO terms from HPO.txt, low-level MPO terms from PhenoGeno and high-level MPO terms from HMD.

<a id="mouse"><h4>MGI-MP associations consolidation from HMD and PhenoGeno</h4></a>

[Back to content](#content)

In [3]:
# dict; key is MGI, value are low-level phenotypes from HMD

f = open(HMD, "r")

mouse_mgi_to_HMD_phenotypes = {}

for num, line in enumerate(f, 1):
    line_data = line.rstrip().split("\t")
    num_fields = len(line_data)
    if num_fields == 4:
        # no HL MP tags
        hl_pheno = None
    else:
        hl_pheno = set(line_data[-1].split(", "))
    hum_symbol = line_data[0]
    hum_etrez = line_data[1]
    mouse_symbol = line_data[2]
    mouse_mgi = line_data[3]

    mouse_mgi_to_HMD_phenotypes[mouse_mgi] = hl_pheno

f.close()

print("MGI with associated phenotypes")
print(len([k for k, v in mouse_mgi_to_HMD_phenotypes.items() if v is not None]))

print("MGI without associated phenotypes")
print(len([k for k, v in mouse_mgi_to_HMD_phenotypes.items() if v is None]))

MGI with associated phenotypes
12577
MGI without associated phenotypes
7554


In [4]:
# dict; key is MGI, value are system-level phenotypes from PhenoGeno

mgi_to_phenogeno = defaultdict(set)

f = open(PHENO_GENO, "r")
for num, line in enumerate(f, 1):
    line_data = line.rstrip().split("\t")
    mp = line_data[3]
    mgis = line_data[5].split("|")
    for mgi in mgis:
        mgi_to_phenogeno[mgi].add(mp)
f.close()

In [5]:
hmd_keys = set(mouse_mgi_to_HMD_phenotypes.keys())
phenogeno_keys = set(mgi_to_phenogeno.keys())

hmd_keys_not_none = {k: v for k, v in mouse_mgi_to_HMD_phenotypes.items() if v is not None}
print(len(hmd_keys))
print(len(phenogeno_keys))
print(len(hmd_keys_not_none.keys()))

hmd_keys_not_none_set = set(hmd_keys_not_none.keys())

20131
22791
12577


In [6]:
mgi_genes_union = reduce(or_, [hmd_keys_not_none_set, phenogeno_keys])
print(len(mgi_genes_union))

22808


In [7]:
# intermediate dataset creation
# dataset contains MGI, MPO terms from HMD, MPO terms from PhenoGeno

f = open("../intermediate_data/mgi_mp_hmd_pg.tsv", "w")
f.write("MGI\tHMD\tPhenoGeno\n")
for mgi in mgi_genes_union:
    hmd_terms = mouse_mgi_to_HMD_phenotypes.get(mgi, None)
    phenogeno_terms = mgi_to_phenogeno.get(mgi, None)

    hmd_terms = hmd_terms if hmd_terms else {"NA"}
    phenogeno_terms = phenogeno_terms if phenogeno_terms else {"NA"}

    hmd_terms = ",".join(hmd_terms)
    phenogeno_terms = ",".join(phenogeno_terms)

    f.write(f"{mgi}\t{hmd_terms}\t{phenogeno_terms}\n")
f.close()

<a id="human"><h4>Human phenotypes dataset consolidation</h4></a>

[Back to content](#content)

In [8]:
# read main table with human and mouse genes into a pandas dataframe

human_mouse_genes = pd.read_csv(HMD, delimiter="\t", header=None)\
    .rename(columns={0: "gene_human", 1: "entrez_id_human", 2: "gene_mouse", 3: "MGI", 4: "MP"}).drop(5, axis=1)

human_HPO_table = pd.read_csv(HPO, delimiter="\t", skiprows=1, header=None, usecols=range(0,4))\
   .rename(columns={0: "entrez_id_human", 1: "entrez-gene-symboln", 2: "HPO-Term-ID", 3: "HPO-Term-Name"})

In [9]:
# combine genes by HPO terms and HPO terms names

human_HPO_table = human_HPO_table\
    .groupby("entrez_id_human")\
    .agg({"HPO-Term-ID": list, "HPO-Term-Name": list})\
    .reset_index()

# concatenate the list of HPOs into a string variable

human_HPO_table["HPO-Term-ID"] = human_HPO_table["HPO-Term-ID"].apply(lambda x: ', '.join(x))
human_HPO_table["HPO-Term-Name"] = human_HPO_table["HPO-Term-Name"].apply(lambda x: ', '.join(x))

# combine the main table with the genes of both mice and humans 
# and the table with the human phenotype by Entrez Gene ID

hpo_db = pd.merge(human_mouse_genes, human_HPO_table, how='left', on = 'entrez_id_human')\
    .rename(columns={"HPO-Term-ID": "HP_low_level"})[["entrez_id_human", "HP_low_level", "MGI"]]

<a id="humandmouse"><h4>Human and mouse datasets combining</h4></a>

Combine ortholodues info (HMD), mouse phenotype info (mgi_mp_hmd_pg.tsv) and human phenotype info (hpo_db).

[Back to content](#content)

In [10]:
mp_db = pd.read_csv("../intermediate_data/mgi_mp_hmd_pg.tsv", delimiter="\t")
hmd_db = pd.read_csv("../databases/HMD.rpt", delimiter="\t", header=None)\
    .rename(columns={0: "gene_human", 1: "entrez_id_human", 2: "gene_mouse", 3: "MGI"})\
    .drop([4, 5], axis=1)

db_with_mp = pd.merge(hmd_db, mp_db, on="MGI", how="left")
db_with_mp_hp = pd.merge(db_with_mp, hpo_db, on=["entrez_id_human", "MGI"], how="left")

In [11]:
db_with_mp_hp = db_with_mp_hp.fillna("[]") # for more convenient to file writing
db_with_mp_hp.head()

,gene_human,entrez_id_human,gene_mouse,MGI,HMD,PhenoGeno,HP_low_level
0,A1BG,1,A1bg,MGI:2152878,[],[],[]
1,A1CF,29974,A1cf,MGI:1917115,"MP:0005378,MP:0005397,MP:0005376,MP:0005367,MP...","MP:0002083,MP:0003222,MP:0003061,MP:0004231,MP...",[]
2,A2M,2,A2m,MGI:2449119,[],[],[]
3,A3GALT2,127550,A3galt2,MGI:2685279,[],[],[]
4,A4GALT,53947,A4galt,MGI:3512453,"MP:0005386,MP:0010768,MP:0005376","MP:0009747,MP:0008874,MP:0009767","HP:0000006, HP:0010970"


In [12]:
# intermediate semi-final dataset with low-level MPO and HPO terms

db_with_mp_hp.to_csv("../intermediate_data/semi_final_db.tsv", sep="\t")

In [13]:
# semi-final dataset written in a proper way

f_in = open("../intermediate_data/semi_final_db.tsv", "r")
f_in.__next__()
f_out = open("../intermediate_data/Human_Mouse_genes_phenotypes_raw.tsv", "w")
f_out.write("gene_human\tentrez_id_human\tgene_mouse\tMGI\tMP_HMD\tMP_PhenoGeno\tHP_genes_to_phenotype\n")

for line in f_in:
    line_data = line.rstrip().split("\t")
    gene_hum = line_data[1]
    entrez_id = line_data[2]
    gene_mouse = line_data[3]
    mgi = line_data[4]
    mp_hmd = line_data[5]
    mp_hmd_to_wr = ",".join(mp_hmd.strip("[]").split(","))
    mp_pg = line_data[6]
    mp_pg_to_wr = ",".join(mp_pg.strip("[]").split(","))
    hp_gtp = line_data[7]
    hp_gtp_to_wr = ",".join(hp_gtp.strip("[]").split(", "))
    f_out.write(f"{gene_hum}\t{entrez_id}\t{gene_mouse}\t{mgi}\t{mp_hmd_to_wr}\t{mp_pg_to_wr}\t{hp_gtp_to_wr}\n")

f_in.close()
f_out.close()

<a id="uplevel"><h4>Up-leveling HPO and MPO terms using HPO-MPO system level term mapping by MGI</h4></a>

Adding to the database columns with system-level HPO and MPO terms and rewriting to file.

[Back to content](#content)

In [14]:
semi_final = pd.read_csv("../intermediate_data/Human_Mouse_genes_phenotypes_raw.tsv", delimiter="\t")

In [23]:
# uploading dataset on HPO to MPO mapping
# dictionary creation (key = HPO term, value = list pf MPO terms)
# empty key-empty value in dict is for convenience for empty fields

mapping_by_mp = pd.read_csv(HP2MP, delimiter="\t")
print(f"HPO terms count {len(set(mapping_by_mp.subject_id.tolist()))}")
print(f"MPO terms count {len(set(mapping_by_mp.object_id.tolist()))}")

mapping_system_level = dict()
for pair in zip(mapping_by_mp.subject_id, mapping_by_mp.object_id):
    if pair[0].strip() not in mapping_system_level.keys():
        mapping_system_level[pair[0].strip()] = [pair[1].strip()]
    else:
        mapping_system_level[pair[0].strip()].append(pair[1].strip())
mapping_system_level[""] = [""]
mapping_system_level

HPO terms count 94
MPO terms count 26


{'HP:0001871': ['MP:0005397'],
 'HP:0004377': ['MP:0002006'],
 'HP:0009124': ['MP:0005375'],
 'HP:0100494': ['MP:0005397', 'MP:0005387'],
 'HP:3000050': ['MP:0005382'],
 'HP:0100685': ['MP:0005390'],
 'HP:0100536': ['MP:0010771'],
 'HP:0100658': ['MP:0005387'],
 'HP:0100881': ['MP:0002006'],
 'HP:0100898': ['MP:0010771'],
 'HP:0001371': ['MP:0005390'],
 'HP:0100790': ['MP:0005378'],
 'HP:0100699': ['MP:0005376'],
 'HP:0000234': ['MP:0005382'],
 'HP:0000464': ['MP:0005378'],
 'HP:0040064': ['MP:0005371'],
 'HP:0001939': ['MP:0005376'],
 'HP:0000816': ['MP:0005384'],
 'HP:0040127': ['MP:0010771', 'MP:0005379'],
 'HP:0025354': ['MP:0005384'],
 'HP:0003565': ['MP:0005397'],
 'HP:0001787': ['MP:0005389'],
 'HP:0001194': ['MP:0005380'],
 'HP:0001560': ['MP:0005380'],
 'HP:0011425': ['MP:0005380'],
 'HP:0001789': ['MP:0005376'],
 'HP:0010880': ['MP:0005380'],
 'HP:0001622': ['MP:0005389'],
 'HP:0002686': ['MP:0005389'],
 'HP:0001557': ['MP:0005386'],
 'HP:0003270': ['MP:0005378'],
 'HP:003014

In [16]:
# creating lists of system-level HPO and MPO terms
# for low2system transpose

hpo_system_level = list(mapping_system_level.keys())
list_mpo = mapping_system_level.values()
mpo_system_level = list(set(list(itertools.chain.from_iterable(list_mpo))))

# for more convenient low2system transpose

semi_final = semi_final.fillna('') 

In [17]:
# adding new columns to semi-final dataset with system HPO and MPO terms

semi_final["MP_system_level"] = semi_final.MP_PhenoGeno\
    .apply(lambda x: replace_terms_by_level(x.split(","), "mp", mpo_system_level))

semi_final["HP_system_level"] = semi_final.HP_genes_to_phenotype\
    .apply(lambda x: replace_terms_by_level(x.split(","), "hp", hpo_system_level))

# adding column with system HPO mapped on system MPO terms

semi_final["MP_from_HP_system_level"] = semi_final.HP_system_level\
    .apply(lambda x: [mapping_system_level[i] for i in x])


In [18]:
semi_final.head()

,gene_human,entrez_id_human,gene_mouse,MGI,MP_HMD,MP_PhenoGeno,HP_genes_to_phenotype,MP_system_level,HP_system_level,MP_from_HP_system_level
0,A1BG,1,A1bg,MGI:2152878,,,,[],[],[[]]
1,A1CF,29974,A1cf,MGI:1917115,"MP:0005378,MP:0005397,MP:0005376,MP:0005367,MP...","MP:0002083,MP:0003222,MP:0003061,MP:0004231,MP...",,"[MP:0005378, MP:0003631, MP:0005397, MP:000537...",[],[[]]
2,A2M,2,A2m,MGI:2449119,,,,[],[],[[]]
3,A3GALT2,127550,A3galt2,MGI:2685279,,,,[],[],[[]]
4,A4GALT,53947,A4galt,MGI:3512453,"MP:0005386,MP:0010768,MP:0005376","MP:0009747,MP:0008874,MP:0009767","HP:0000006,HP:0010970","[MP:0005386, MP:0010768, MP:0005376]",[HP:0001871],[[MP:0005397]]


In [19]:
semi_final.to_csv("../intermediate_data/semi_final_system.tsv", sep="\t")

In [20]:
# final dataset writing in a covenient format without brackets and quotes

f_in = open("../intermediate_data/semi_final_system.tsv", "r")
f_in.__next__()
f_out = open("../databases/human_mouse_GPO.tsv", "w")
f_out.write("gene_human\tentrez_id_human\tgene_mouse\tMGI\tMP_HMD\tMP_PhenoGeno\tHP_genes_to_phenotype\tMP_system_level\tHP_system_level\tMP_from_HP_system_level\n")

for line in f_in:
    line_data = line.rstrip().split("\t")
    gene_hum = line_data[1]
    entrez_id = line_data[2]
    gene_mouse = line_data[3]
    mgi = line_data[4]
    mp_hmd = line_data[5]
    mp_pg = line_data[6]
    hp_gtp = line_data[7]
    mp_syst_lvl = ",".join([i.strip("'") for i in line_data[8].strip("[]").split(", ")])
    hp_syst_lvl = ",".join([i.strip("'") for i in line_data[9].strip("[]").split(", ")])
    mp_from_hp_sysl_lvl = ",".join(set([i.strip("[]'") for i in line_data[10][1:-1].split(", ")]))
    f_out.write(f"{gene_hum}\t{entrez_id}\t{gene_mouse}\t{mgi}\t{mp_hmd}\t{mp_pg}\t{hp_gtp}\t{mp_syst_lvl}\t{hp_syst_lvl}\t{mp_from_hp_sysl_lvl}\n")

f_in.close()
f_out.close()

<a id="mono"><h4>Filtering the dataset to monosemantic</h4></a>

Gene filtering so that only monosemantic genes (having only one orthologue in each organism) are present.

[Back to content](#content)

In [4]:
gpo = pd.read_csv(
    "../databases/human_mouse_GPO.tsv", delimiter="\t"
)

gpo.head()

,gene_human,entrez_id_human,gene_mouse,MGI,MP_HMD,MP_PhenoGeno,HP_genes_to_phenotype,MP_system_level,HP_system_level,MP_from_HP_system_level
0,A1BG,1,A1bg,MGI:2152878,NaN,NaN,NaN,NaN,NaN,NaN
1,A1CF,29974,A1cf,MGI:1917115,"MP:0005378,MP:0005397,MP:0005376,MP:0005367,MP...","MP:0002083,MP:0003222,MP:0003061,MP:0004231,MP...",NaN,"MP:0005378,MP:0003631,MP:0005397,MP:0005376,MP...",NaN,NaN
2,A2M,2,A2m,MGI:2449119,NaN,NaN,NaN,NaN,NaN,NaN
3,A3GALT2,127550,A3galt2,MGI:2685279,NaN,NaN,NaN,NaN,NaN,NaN
4,A4GALT,53947,A4galt,MGI:3512453,"MP:0005386,MP:0010768,MP:0005376","MP:0009747,MP:0008874,MP:0009767","HP:0000006,HP:0010970","MP:0005386,MP:0010768,MP:0005376",HP:0001871,MP:0005397


In [19]:
# Filtering the database by monosemantic feature and
# Creating the monosemantic dataset

gene_count_human = (
    pd.DataFrame(gpo.entrez_id_human.value_counts())
    .reset_index()
    .rename(columns={"count": "count_human"})
)

gene_count_mouse = (
    pd.DataFrame(gpo.MGI.value_counts())
    .reset_index()
    .rename(columns={"count": "count_mouse"})
)

gpo_gene_count = gpo.merge(
    gene_count_human, on="entrez_id_human", how="left"
).merge(gene_count_mouse, on="MGI", how="left")

gpo_mono = gpo_gene_count[
    (gpo_gene_count.count_human == 1) & (gpo_gene_count.count_mouse == 1)
]

gpo_mono = gpo_mono.drop(["count_human", "count_mouse"], axis=1)

gpo_mono.to_csv("../databases/human_mouse_GPO_mono.tsv", sep="\t", index=False)

print(f"human_mouse_GPO monosematic: {gpo_mono.shape[0]}")

human_mouse_GPO monosematic: 16985


<a id="lowfilter"><h4>Low-level MPO and HPO terms filtering</h4></a>

Filtering MPO and HPO low-level terms so that only children of upper-level terms used in HPO-MPO mapping by MGI are present.

[Back to content](#content)

In [3]:
hpo = OntologyFactory().create(handle=PATH_HPO)
mpo = OntologyFactory().create(handle=PATH_MPO)

gpo_mono = pd.read_csv("../databases/human_mouse_GPO_mono.tsv", sep="\t")

In [6]:
# Creating lists of unique MPO and HPO system terms that present in MGI mapping

# Lists of unique upper-level MPO and HPO terms from mapping
hpo_terms = list(set([term.strip() for term in mapping_by_mp.subject_id.tolist()]))
mpo_terms = list(set([term.strip() for term in mapping_by_mp.object_id.tolist()]))

# Lists of all descendants for mapped upper-level terms
mpo_lower_terms = list()
hpo_lower_terms = list()

# Add upper-level MPO and HPO terms to the list
mpo_lower_terms.extend(mpo_terms)
hpo_lower_terms.extend(hpo_terms)

for term in mpo_terms:
    mpo_lower_terms.extend(mpo.descendants(term))

for term in hpo_terms:
    hpo_lower_terms.extend(hpo.descendants(term))

# Sets with unique lower-level terms that are descendants of the mapped upper-level ones
mpo_lower = set(mpo_lower_terms)
hpo_lower = set(hpo_lower_terms)

In [7]:
def mapped_low(terms_initial, terms_allowed):
    """
    Takes string of initial terms, comma without space separated,
    set of allowed terms
    returns string of comma without space separated 
    terms that lie in the intersection of initial and allowed terms 
    """
    terms_initial = set(terms_initial.split(","))
    terms_allowed = set(terms_allowed)
    terms_result = ",".join(list(terms_initial.intersection(terms_allowed)))
    return terms_result

In [8]:
def isnan(x):
    return x != x

In [9]:
gpo_mono["MPO_lower"] = gpo_mono.MP_PhenoGeno\
    .apply(lambda x: mapped_low(x, mpo_lower_terms) if not isnan(x) else x)
gpo_mono["HPO_lower"] = gpo_mono.HP_genes_to_phenotype\
    .apply(lambda x: mapped_low(x, hpo_lower_terms) if not isnan(x) else x)

gpo_mono.to_csv("../databases/human_mouse_GPO_v3.tsv", sep="\t", index=False)

In [14]:
gpo_mono.head()

,gene_human,entrez_id_human,gene_mouse,MGI,MP_HMD,MP_PhenoGeno,HP_genes_to_phenotype,MP_system_level,HP_system_level,MP_from_HP_system_level,MPO_lower,HPO_lower
0,A1BG,1,A1bg,MGI:2152878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1CF,29974,A1cf,MGI:1917115,"MP:0005370,MP:0005376,MP:0005385,MP:0005397,MP...","MP:0006142,MP:0002833,MP:0004090,MP:0004084,MP...",NaN,"MP:0005370,MP:0005376,MP:0005385,MP:0005389,MP...",NaN,NaN,"MP:0010754,MP:0010052,MP:0008772,MP:0006142,MP...",NaN
2,A2M,2,A2m,MGI:2449119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A3GALT2,127550,A3galt2,MGI:2685279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A4GALT,53947,A4galt,MGI:3512453,"MP:0010768,MP:0005376,MP:0005386","MP:0009747,MP:0008874,MP:0009767","HP:0000006,HP:0010970","MP:0010768,MP:0005376,MP:0005386",HP:0001871,MP:0005397,"MP:0009747,MP:0008874,MP:0009767",HP:0010970


In [12]:
# How many genes have not allowed low-level MPO terms

(gpo_mono.MP_PhenoGeno\
    .apply(lambda x: len(x) if not isnan(x) else x) 
 == gpo_mono.MPO_lower\
    .apply(lambda x: len(x) if not isnan(x) else x)).sum()

10145

In [13]:
# How many genes have not allowed low-level MPO terms

(gpo_mono.HP_genes_to_phenotype\
    .apply(lambda x: len(x) if not isnan(x) else x) 
 == gpo_mono.HPO_lower\
    .apply(lambda x: len(x) if not isnan(x) else x)).sum()

213